In [1]:
!conda install --yes --quiet altair psycopg2



# All requested packages already installed.
# packages in environment at /opt/conda:
#
altair                    1.2.1                      py_0    conda-forge
psycopg2                  2.7.3.2                  py36_0    conda-forge


In [2]:
%run __init__.py

In [3]:
!pip install spacy nltk

In [4]:
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 1.2MB/s ta 0:00:011

    Linking successful
    /opt/conda/lib/python3.6/site-packages/en_core_web_sm -->
    /opt/conda/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [25]:
%run nlp.py

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_pickle('../pickled_data/wiki_data_clean_text.pkl')

In [7]:
df['page_text_len'] = df.page_text_clean.map(lambda x: len(x.split()))

In [8]:
df.page_text_len.sort_values(ascending=False).head(10)

913     9518
2873    7708
916     6975
231     6916
249     6577
2147    6509
3536    5830
96      5662
671     5230
891     5067
Name: page_text_len, dtype: int64

## Label encode the category

In [9]:
le = LabelEncoder()
df['ult_parent_numerical'] = le.fit_transform(df['ult_parent'])

## Prepare Document Term Matrix

In [10]:
tfidf_vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1, 1))
document_term_matrix_sps = tfidf_vectorizer.fit_transform(df.page_text_clean)

In [11]:
document_term_matrix_sps.shape

(4099, 51643)

In [12]:
document_term_matrix_df = pd.DataFrame(document_term_matrix_sps.toarray(),
                                       index=df.page_title,
                                       columns=tfidf_vectorizer.get_feature_names())

In [13]:
document_term_matrix_df.head()

,aa,aaa,aaaa,aaaaj,aaabc,aaabcaaabcaaabcaaabc,aaai,aaajxb,aaas,aab,...,zylab,zynga,zynx,zynxambulatory,zynxcare,zynxevidence,zynxorder,zyrion,zz,zzap
page_title,,,,,,,,,,,,,,,,,,,,,
Data exploration,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
List of datasets for machine learning research,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Machine learning,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Outline of machine learning,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Accuracy paradox,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Search

In [14]:
wiki_keywords = [
    {'keyword': 'lasso'},
    {'keyword': 'Excel'},
    {'keyword': 'Oracle'},
    {'keyword': 'support vector machine'},
    {'keyword': 'logistic regression'},
    {'keyword': 'natural language processing'},
    {'keyword': 'TurboTax'}
]

wiki_keywords_df = pd.DataFrame(wiki_keywords)
wiki_keywords_df

,keyword
0,lasso
1,Excel
2,Oracle
3,support vector machine
4,logistic regression
5,natural language processing
6,TurboTax


In [15]:
keywords_encoded = tfidf_vectorizer.transform(wiki_keywords_df.keyword)

In [16]:
keywords_encoded_df = pd.DataFrame(keywords_encoded.toarray(), 
                                       index=wiki_keywords_df.keyword, 
                                       columns=tfidf_vectorizer.get_feature_names())

In [17]:
keywords_encoded_df.sample(4)

,aa,aaa,aaaa,aaaaj,aaabc,aaabcaaabcaaabcaaabc,aaai,aaajxb,aaas,aab,...,zylab,zynga,zynx,zynxambulatory,zynxcare,zynxevidence,zynxorder,zyrion,zz,zzap
keyword,,,,,,,,,,,,,,,,,,,,,
TurboTax,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oracle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Excel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
support vector machine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Select a Random Keyword

In [18]:
random_keyword_df = keywords_encoded_df.sample()
random_keyword_df

,aa,aaa,aaaa,aaaaj,aaabc,aaabcaaabcaaabcaaabc,aaai,aaajxb,aaas,aab,...,zylab,zynga,zynx,zynxambulatory,zynxcare,zynxevidence,zynxorder,zyrion,zz,zzap
keyword,,,,,,,,,,,,,,,,,,,,,
Excel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Append the search term to the document term matrix

In [19]:
dtm_with_search_term = document_term_matrix_df.append(random_keyword_df)

## Compute SVD of Augmented Document Term Matrix

In [20]:
n_components = 50
SVD = TruncatedSVD(n_components)
component_names = ["component_"+str(i+1) for i in range(n_components)]

In [21]:
svd_matrix = SVD.fit_transform(dtm_with_search_term)

In [22]:
svd_df = pd.DataFrame(svd_matrix, 
                      index=dtm_with_search_term.index, 
                      columns=component_names)

## Identify the Vector for our Search Term

In [24]:
search_term_svd_vector = svd_df.loc[random_keyword_df.index]
search_term_svd_vector

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,component_41,component_42,component_43,component_44,component_45,component_46,component_47,component_48,component_49,component_50
Excel,0.014904,-0.008866,-0.00738,-0.009253,-0.030398,0.004078,-0.030654,0.037165,-0.009811,-0.00455,...,0.024188,-0.040488,0.017002,0.079302,-0.0408,-0.000398,0.048318,0.029099,0.004722,0.020905


## Use Cosine Similarity to Find the Most Similar Vector

In [26]:
svd_df['cosine_sim'] = cosine_similarity(svd_df, search_term_svd_vector)

In [27]:
svd_df[['cosine_sim']].sort_values('cosine_sim', ascending=False).head(5)

,cosine_sim
Excel,1.000000
LibreOffice Calc,0.821157
FarPoint Spread,0.801284
Gnumeric,0.797423
Logical spreadsheet,0.780991
